# Mental illness at work

Negli ultimi vent’ anni, la nostra società ha subito un radicale cambiamento, il passaggio da una società “moderna” ad una società “post-moderna" è stato veloce e sconvolgente. Oggi si parla di società digitale, di cyberspazio, di e-commerce , di Internet o di società "fluida" e "multi-rete”.

L’avvento tecnologico ha senza dubbio rivoluzionato il nostro modo di vivere, portando molteplici vantaggi, ma spesso ci si dimentica che esistono anche degli aspetti negativi legati all’espansione di questo settore, legati all’abuso e al modo errato di servirsi delle tecnologie a nostra disposizione.

Diversi studi hanno evidenziato che molte  problematiche di carattere neuropsicologico sono connesse all’utilizzo errato delle nuove tecnologie.
Durante la 18esima edizione del Congresso Mondiale di Psichiatria dinamica di Firenze, tenuto nell’Aprile 2017, gli esperti hanno precisato che le dipendenze tecnologiche sono le malattie mentali destinate a diffondersi maggiormente a livello mondiale nei prossimi dieci anni. In Italia, a seguito di analisi condotte su un campione di lavoratori operanti nel settore tecnologico, è emersa la presenza di disturbi neuropsicologici per il 49% dei casi, tra cui quelli legati al sonno e al calo di attenzione, seguiti da depressione nel 28% dei casi, e ansia nel 23% dei casi.

Nell’attuale fase storica, definita era digitale, assistiamo ad un continuo aumento della domanda di nuove e sempre più sofisticate tecnologie.
In questo report si è scelto di analizzare i lati negativi di questa espansione, non dal punto di vista del consumatore, ma da quello di chi ha scelto di far diventare di questa avanguardia la propria professione.
 
Il dataset analizzato presenta 1433 osservazioni, raccolte tramite un questionario in rete, in cui sono presenti informazioni riguardanti i disturbi psichici, il loro trattamento all’interno delle aziende, il modo in cui questi influenzano l’attività lavorativa e la frequenza con cui si presentano in questo settore.


## Loading the dataset

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Caricamento del dataset
import pandas as pd
import numpy as np

df = pd.read_csv("survey.csv")

# Importazione della libreria plotly
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

# Importazione del plugin per disegnare le HeatMap
import folium
from folium.plugins import HeatMap

## Age plot - Histogram

Il grafico seguente rappresenta la distribuzione dell'attributo "age" all'interno del dataset.
Ogni barra del diagramma corrisponde ad un'età differente. Considerando che il campione di persone su cui è stata effettuata l'analisi è composto unicamente da individui che lavorano nel settore tecnologico, è facile pensare che la frequenza di picco si presenterà tra i 25 e i 40 anni. Si osserva che all'aumentare dell'età vi è una graduale diminuzione di frequenza, giustificata dal fatto che ci si avvicina all'età di pensionamento. Nella fascia registrata prima dei 25 anni compaiono un minor numero di osservazioni, in quanto è il periodo in cui, nella maggior parte dei casi, ci si dedica allo studio.

In [2]:
#Caricamento dei dati relativi al campo: age
age = df['What is your age?'].values
ageSet = set(age)

#Rimozione degli outliers
age = age[(age <= 90) & (age >= 16)]

layout = go.Layout(
    title='Age of respondents',
    xaxis=dict(
        title='age of people'
    ),
    yaxis=dict(
        title='number of people'
    ),
    bargap=0.1,
)

data = [go.Histogram(x=age)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

## Country plot - Geographical heatmap

Il grafico seguente rappresenta la distribuzione geografica delle osservazioni raccolte. Si evidenzia una concentrazione elevata di risposte negli Stati Uniti, in quanto il sondaggio attraverso il quale sono stati raccolti i dati è stato pubblicato in questo paese; inoltre, è risaputo che la zona geografica della Silicon Valley è popolata da molte persone e aziende che operano nel settore informatico.

In [3]:
#Caricamento dei dati relativi al campo: countryLive
countryLive = df['What country do you live in?'].values
countryLiveSet = set(countryLive)

dfCoordinates = pd.read_csv("coordinates.csv")

country = dfCoordinates['Country'] 
lat = dfCoordinates['lat']
lon = dfCoordinates['lon']

counters = np.zeros(len(country))

for c in countryLive:
    for i in range(len(country)):
        if(c == country[i]):
            counters[i] = counters[i] + 1
            
hmap = folium.Map(location=[45, 25], zoom_start=2 )

hm_wide = HeatMap( list(zip(lat, lon, counters)),
                   min_opacity=0.2,
                   max_val=1000,
                   radius=25, blur=20, 
                   max_zoom=1, 
                 )


hmap.add_child(hm_wide)

## Gender plot - Histogram

Il grafico seguente rappresenta la distribuzione dell'attributo "gender". La percentuale maschile è nettamente più alta, confermata da una reale disparità di genere in questo ambito.

In [4]:
#Caricamento dei dati relativi al campo: gender
gender = df['What is your gender?'].dropna()
gender = np.array(gender)
genderSet = set(gender)

#Pulizia dei dati
mask = []
for i in range(len(gender)):
    value = type(gender[i]) == type("str")
    mask.append(value)

mask = np.array(mask, dtype = bool)
gender = gender[mask]

for i in range(len(gender)):
    gender[i] = gender[i].lower()
    
male = 0
female = 0
other = 0

for i in range(len(gender)):
    if (("female" in gender[i])or(gender[i] == "f")or(gender[i] == "woman")):
        gender[i] = "female"
        female = female + 1
    elif ("male" in gender[i])or (gender[i] == "m")or(gender[i] == "man"):
        gender[i] = "male"
        male = male + 1
    else:
        gender[i] = "other"
        other = other + 1
    
other = (other*100)/len(gender)
female = (female*100)/len(gender)
male = (male*100)/len(gender)

layout = go.Layout(
    title='Gender of respondents',
    xaxis=dict(
        title='number of people %'
    ),
    bargap=0.1,
)

trace1 = go.Bar(
    y=["Other ","Female ","Male "],
    x=[other,female,male],
    orientation='h',
)

data = [trace1]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

## Work role plot - Histogram

In questo grafico si osservano le posizioni lavorative predominanti. Al primo posto vi è lo sviluppo web.

In [5]:
#Caricamento dei dati relativi al campo: workPosition
workPosition = df['Which of the following best describes your work position?'].values
workPositionLower = [w.lower() for w in workPosition]
workPositionSet = set(workPositionLower)

wp = np.array([])
for w in workPosition:
    wp = np.concatenate((wp,np.array(w.split('|'))), axis = None )
wpSet = np.unique(wp)

counters = np.zeros(len(wpSet))

i = 0
for el in wpSet:
    for v in wp:
        if(v == el):
            counters[i] = counters[i]+1
    i = i + 1

indice = np.argsort(counters)

countersWP = counters[indice]
wpSet = wpSet[indice]

countersWP = (countersWP*100)/len(wp)

layout = go.Layout(
    title='Work position of respondents',
    xaxis=dict(
        title='number of people %'
    ),
    bargap=0.1,
    yaxis=dict(
         automargin=True
    ),
)

trace1 = go.Bar(
    y=wpSet,
    x=countersWP,
    orientation='h'
)

data = [trace1] 

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

## Mental illness frequency plot - Histogram

Questo grafico rileva quali siano i disturbi mentali più frequenti all'interno del settore tecnologico. Si osserva che la maggioranza dei lavoratori che riscontra un disturbo mentale soffre di sbalzi di umore o di ansia.

In [6]:
#Caricamento dei dati relativi al campo: conditionDiagnosedByMedical
conditionDiagnosedByMedical = df['If so, what condition(s) were you diagnosed with?'].dropna()
conditionDiagnosedByMedicalSet = set(conditionDiagnosedByMedical)

cdbm = np.array([])
for el in conditionDiagnosedByMedical:
    cdbm = np.concatenate((cdbm,np.array(el.split('|'))), axis = None )
cdbmSet = np.unique(cdbm)

counters = np.zeros(len(cdbmSet))

i = 0
for el in cdbmSet:
    for v in cdbm:
        if(v == el):
            counters[i] = counters[i]+1
    i = i + 1

mask = counters >= 10

somma = np.sum(mask)
altro = len(mask) - somma

cdbmSet = cdbmSet[mask]
counters = counters[mask]

counters = list(counters)
cdbmSet = list(cdbmSet)
counters.append(altro)
cdbmSet.append("Other")
counters = np.array(counters)
cdbmSet = np.array(cdbmSet)

indice = np.argsort(counters)
counters = counters[indice]
cdbmSet = cdbmSet[indice]

countersMH = (counters*100)/len(cdbm)

layout = go.Layout(
    title='Mental illness frequency',
    xaxis=dict(
        title='number of people %'
    ),
    bargap=0.1,
    yaxis=dict(
         automargin=True
    )
)

data = [go.Bar(
            y=cdbmSet,
            x=countersMH,
            orientation='h'
    )]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

## Mental illness frequency per role plot - Heatmap

Il grafico seguente mostra la relazione tra i disturbi mentali e le posizioni lavorative.
Si osserva una distrubuzione piuttosto omogenea dei dati, perciò non è possibile affermare che vi siano posizioni lavorative che inducano ad un maggior rischio di sviluppo di una malattia mentale.
E' tuttavia interessante notare come la frequenza di osservazioni sia accentuata nell'ambito del settore HR. Questo fenomeno potrebbe essere giustificato dal fatto che spesso chi lavora in questo ambito è sottoposto a situazioni stressanti come, ad esempio, la valutazione di altre persone, l'inserimento di neoassunti nell'azienda e/o la riduzione del personale.
In generale si nota una predisposizione dei lavoratori a sviluppare disturbi dell’umore e in misura minore d'ansia.

In [7]:
#Caricamento dei dati relativi al campo: conditionDiagnosedByMedical
conditionDiagnosedByMedical = df['If so, what condition(s) were you diagnosed with?']
conditionDiagnosedByMedicalSet = set(conditionDiagnosedByMedical)

#Caricamento dei dati relativi al campo: workPosition
workPosition = df['Which of the following best describes your work position?'].values
workPositionLower = [w.lower() for w in workPosition]
workPositionSet = set(workPositionLower)

wp = np.array([])
for w in workPosition:
    wp = np.concatenate((wp,np.array(w.split('|'))), axis = None )
wpSet = np.unique(wp)

listaf = []
totale = 0
for k in range(len(cdbmSet)):
    malattiaDati = np.zeros(len(wpSet))
    for j in range(len(wpSet)):
        for i in range(len(conditionDiagnosedByMedical)):
            if(type(conditionDiagnosedByMedical[i]) == type("str")):
                if(cdbmSet[k] in conditionDiagnosedByMedical[i])and(wpSet[j] in workPosition[i]):
                    malattiaDati[j] = malattiaDati[j] + 1
    totale = totale + np.sum(malattiaDati)
    listaf.append(list(malattiaDati))
            
listaf = np.array(listaf)
for i in range(len(wpSet)):
    listaf[:,i] = ((listaf[:,i]*100)/np.sum(listaf[:,i]))

trace = go.Heatmap(z=listaf,
                   x=wpSet,
                   y=cdbmSet)
data=[trace]
layout = go.Layout(
    title='Mental illness frequency per work position',
    yaxis=dict(
         automargin=True
    ),
    xaxis=dict(
         automargin=True
    )
    
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')

## Number of ills people plot - Pie

Il grafico seguente indica la percentuale di persone malate, non malate, o che pensano di esserlo ma non si sono sottoposte ad un controllo specialistico.

In [8]:
#Caricamento dei dati relativi al campo: currentMentalDisorder
currentMentalDisorder = df['Do you currently have a mental health disorder?'].values
currentMentalDisorderSet = set(currentMentalDisorder)

counters = np.zeros(len(currentMentalDisorderSet))
i = 0
for el in currentMentalDisorderSet:
    for v in currentMentalDisorder:
        if(v == el):
            counters[i] = counters[i]+1
    i = i + 1

counters = list(counters)
labels = np.unique(currentMentalDisorder)
labels = list(labels)

layout = go.Layout(title='Do you currently have a mental health disorder?')
trace = go.Pie(labels=labels, values=counters, marker=dict(line=dict(color='rgb(0, 0, 0)', width=2)))

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='styled histogram')

## Geographical distribution of ills people plot - Geographical heatmap

Il grafico seguente rappresenta la distribuzione geografica delle persone con disturbi mentali. Eseguendo un confronto con la medesima tipologia di grafico, sviluppato sull'intera popolazione del dataset, si osserva che i malati si distribuiscono equamente sul territorio mondiale.

In [9]:
#Caricamento dei dati relativi al campo: countryLive
countryLive = df['What country do you live in?'].values
countryLiveSet = set(countryLive)

#Caricamento dei dati relativi al campo: diagnosedByMedical
diagnosedByMedical = df['Have you been diagnosed with a mental health condition by a medical professional?'].values
diagnosedByMedicalSet = set(diagnosedByMedical)

dfCoordinates = pd.read_csv("coordinates.csv")

country = dfCoordinates['Country'] 
lat = dfCoordinates['lat']
lon = dfCoordinates['lon']

counters = np.zeros(len(country))

for j in range(len(countryLive)):
    for i in range(len(country)):
        if(countryLive[j] == country[i])&(diagnosedByMedical[j]=="Yes"):
            counters[i] = counters[i] + 1
            
hmap = folium.Map(location=[45, 25], zoom_start=2 )

hm_wide = HeatMap( list(zip(lat, lon, counters)),
                   min_opacity=0.2,
                   max_val=700,
                   radius=25, blur=20, 
                   max_zoom=1, 
                 )


hmap.add_child(hm_wide)

## Company number of employees plot - Pie

Il diagramma a torta seguente fornisce informazioni relative alla dimensione delle aziende in cui lavorano le persone presenti nel dataset. Si osserva che circa il 50% degli individui su cui è basata l'analisi fa parte di compagnie con oltre 100 dipendenti.

In [10]:
#Caricamento dei dati relativi al campo: nEmployees
nEmployees = df['How many employees does your company or organization have?']
nEmployees = nEmployees.dropna()
nEmployeesSet = np.unique(nEmployees)

counters = np.zeros(len(nEmployeesSet))
i = 0
for el in nEmployeesSet:
    for v in nEmployees:
        if(v == el):
            counters[i] = counters[i]+1
    i = i + 1
    
labels = nEmployeesSet

indice = [5,3,1,2,4,0]
labels = labels[indice]
counters = counters[indice]

trace = go.Pie(labels=labels, values=counters)

layout = go.Layout(title='How many employees does your company or organization have?')
trace = go.Pie(labels=labels, values=counters, marker=dict(line=dict(color='rgb(0, 0, 0)', width=2)), sort=False)

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='styled histogram')

## Protection of anonymity - Stacked bar

I grafici a seguire mostrano come le aziende promuovono iniziative per aiutare i dipendenti che soffrono di tali patologie.
I risultati sono scoraggianti: dall'analisi si evince che le persone non sanno se possa essere mantenuta la riservatezza riguardo a tali problematiche in ambito lavorativo.
In particolar modo, il 26 % dei rispondendi afferma che il proprio datore di lavoro attribuisce minor rilevanza ai problemi mentali rispetto a quelli fisici.
Un altro aspetto preoccupante è che il 65 % dei lavoratori non conosce i propri diritti alla privacy in ambiente lavorativo.

In [11]:
#Caricamento dei dati relativi al campo: anonymity
anonymity = df['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'].dropna()
anonymitySet = set(anonymity)

#Caricamento dei dati relativi al campo: mentalVSPhysical
mentalVSPhysical = df['Do you feel that your employer takes mental health as seriously as physical health?'].dropna()
mentalVSPhysicalSet = set(mentalVSPhysical)

def getResponse(a):
    response = np.zeros(len(set(a)))
    for e in a:
        if(e == "Yes"):
            response[0] = response[0] + 1
        elif(e == "I don't know"):
            response[1] = response[1] + 1
        else:
            response[2] = response[2] + 1
    response = (response*100)/len(a)
    return response

anonymityData = getResponse(anonymity)
mentalVSPhysicalData = getResponse(mentalVSPhysical)

top_labels = ['Yes', "I don't know", 'No']

colors = ['rgb(83, 198, 83)', 'rgb(255, 219, 77)',
          'rgb(255, 133, 102)']

x_data = [anonymityData,mentalVSPhysicalData]

y_data = ['Is your anonymity protected if <br>you choose to take advantage of <br>mental health or substance<br> abuse treatment resources <br>provided by your employer?',
          'Do you feel that your employer<br>takes mental health as seriously<br>as physical health?']

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    title = "How are mental illnesses treated at work?",
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str("{0:.0f}".format(xd[0])) + '%',
#                             text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str("{0:.0f}".format(xd[i])) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='bar-colorscale')


## Discuss your pathology - Stacked bar

Il grafico a seguire mostra in che misura il parlare delle proprie problematiche a colleghi e datori di lavoro possa essere d'aiuto. Si evince che una buona parte delle persone affette da questi disturbi trovi conforto nel confidarsi con i propri colleghi e diretti superiori.
Si evidenzia inoltre un dato negativo: in generale, si pensa che parlare della propria malattia mentale con il datore di lavoro possa in qualche modo avere delle conseguenze negative. Così non è, invece, per quanto riguarda le malattie fisiche.

In [12]:
#Caricamento dei dati relativi al campo: discussingMentalConsequences
discussingMentalConsequences = df['Do you think that discussing a mental health disorder with your employer would have negative consequences?'].dropna()
discussingMentalConsequencesSet = set(discussingMentalConsequences)

#Caricamento dei dati relativi al campo: discussingPhysicalConsequences
discussingPhysicalConsequences = df['Do you think that discussing a physical health issue with your employer would have negative consequences?'].dropna()
discussingPhysicalConsequencesSet = set(discussingPhysicalConsequences)

#Caricamento dei dati relativi al campo: confortCoworkersDiscussing
confortCoworkersDiscussing = df['Would you feel comfortable discussing a mental health disorder with your coworkers?'].dropna()
confortCoworkersDiscussingSet = set(confortCoworkersDiscussing)

#Caricamento dei dati relativi al campo: confortSupervisorDiscussing
confortSupervisorDiscussing = df['Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?'].dropna()
confortSupervisorDiscussingSet = set(confortSupervisorDiscussing)

def getResponse(a):
    response = np.zeros(len(set(a)))
    for e in a:
        if(e == "Yes"):
            response[0] = response[0] + 1
        elif(e == "Maybe"):
            response[1] = response[1] + 1
        else:
            response[2] = response[2] + 1
    response = (response*100)/len(a)
    return response

discussingMentalConsequencesData = getResponse(discussingMentalConsequences)
discussingPhysicalConsequencesData = getResponse(discussingPhysicalConsequences)
confortCoworkersDiscussingData = getResponse(confortCoworkersDiscussing)
confortSupervisorDiscussingData = getResponse(confortSupervisorDiscussing)

top_labels = ['Yes', "Maybe", 'No']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)']

x_data = [discussingMentalConsequencesData,discussingPhysicalConsequencesData,confortCoworkersDiscussingData,confortSupervisorDiscussingData]

y_data = ['discussing a mental health<br>disorder with your employer would <br>have negative consequences?',
          'discussing a physical health<br>disorder with your employer would <br>have negative consequences?',
          'Would you feel comfortable<br> discussing a mental health <br>disorder with your coworkers?',
          'Would you feel comfortable<br> discussing a mental health <br>disorder withyour direct supervisor?',]

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    title='What are the consequences of talking about your pathology?',
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str("{0:.0f}".format(xd[0])) + '%',
#                             text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str("{0:.0f}".format(xd[i])) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='bar-colorscale')


## Influence of mental illnesses on productivity- Histograms

I grafici a seguire riportano dati relativi al grado in cui la malattia interferisce con la produttività dell'individuo.
L'80 % dei lavoratori afferma che la propria capacità lavorativa è influenzata dalla patologia mentale:
- 45 % Lievemente influenzata
- 35 % Mediamente influenzata
- 20 % Molto influenzata

In [13]:
#Caricamento dei dati relativi al campo: productivity
productivity = df['Do you believe your productivity is ever affected by a mental health issue?'].dropna()
productivitySet = set(productivity)

productivity = np.array(productivity)
mask = []

#Eliminazione delle persone che non soffrono di alcuna patologia
for el in productivity:
    mask.append(el != "Not applicable to me")
    
mask = np.array(mask, dtype = bool)
productivity = productivity[mask]
productivitySet = np.unique(productivity)

counters = np.zeros(len(productivitySet))
i = 0
for el in productivitySet:
    for v in productivity:
        if(v == el):
            counters[i] = counters[i]+1
    i = i + 1

layout = go.Layout(title='Do you believe your productivity is ever affected by a mental health issue?')
trace = go.Pie(labels=productivitySet, values=counters, marker=dict(line=dict(color='rgb(0, 0, 0)', width=2)))

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='styled histogram')

####################################################################################################################################################################################################################################

#Caricamento dei dati relativi al campo: percentageWorkAffected
percentageWorkAffected = df['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'].dropna()
percentageWorkAffectedSet = set(percentageWorkAffected)

def getResponse(a):
    response = np.zeros(len(set(a)))
    for e in a:
        if(e == "1-25%"):
            response[0] = response[0] + 1
        elif(e == "26-50%"):
            response[1] = response[1] + 1
        elif(e == "51-75%"):
            response[2] = response[2] + 1
        else:
            response[3] = response[3] + 1
    response = (response*100)/len(a)
    return response

whatPercentData = getResponse(percentageWorkAffected)

top_labels = ['Low', "Medium-low", 'Medium-high','High']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)','rgba(130, 128, 180, 0.8)']

x_data = [whatPercentData]

y_data = ['If yes, what percentage of your <br> work time (time performing primary<br> or secondary job functions) is <br>affected by a mental health issue?']


traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str("{0:.0f}".format(xd[0])) + '%',
#                             text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str("{0:.0f}".format(xd[i])) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='bar-colorscale')

## Correlation between employee work and mental illness - Contingency table, Stacked bar


Il grafico seguente mostra che non vi è alcuna relazione tra il fatto di essere un lavoratore dipende o autonomo e di aver riscontrato una malattia mentale.

In [14]:
#Caricamento dei dati relativi al campo: self_employed
self_employed = df['Are you self-employed?']
selfEmployedSet = set(self_employed)

self_employedYesNo = np.full(self_employed.shape,"",dtype="object")
for i in range(len(self_employed)):
    if(self_employed[i]==0):
        self_employedYesNo[i] = "not self employed"
    else:
        self_employedYesNo[i] = "self employed"

#Caricamento dei dati relativi al campo: diagnosedByMedical
diagnosedByMedical = df['Have you been diagnosed with a mental health condition by a medical professional?'].values
diagnosedByMedicalYesNo = np.full(diagnosedByMedical.shape,"",dtype="object")
for i in range(len(diagnosedByMedical)):
    if(diagnosedByMedical[i]=='No'):
        diagnosedByMedicalYesNo[i] = "MH not diagnosed"
    else:
        diagnosedByMedicalYesNo[i] = "MH diagnosed"
setSelf_employedYesNo = np.unique(self_employedYesNo)
setDiagnosedByMedicalYesNo = np.unique(diagnosedByMedicalYesNo)
table = np.full((len(setSelf_employedYesNo), len(setDiagnosedByMedicalYesNo)), np.nan, dtype=int)
for i in range (len(setSelf_employedYesNo)):
    for j in range (len(setDiagnosedByMedicalYesNo)):
        table[i,j] = np.sum((self_employedYesNo==setSelf_employedYesNo[i]) & (diagnosedByMedicalYesNo==setDiagnosedByMedicalYesNo[j]))

freq = np.sum(table, axis=1, keepdims=True) # somma i dipendenti e i lavoratori in proprio
people = freq[0]+freq[1]

t_cond_rows = table / freq

MNS = t_cond_rows[0][0]*100
MS = t_cond_rows[1][0]*100

NMNS = t_cond_rows[0][1]*100
NMS = t_cond_rows[1][1]*100

print(f"\nNot Self employed and MH diagnosed: {MNS:.2f} %")
print(f"\nNot Self employed and MH not diagnosed: {NMNS:.2f} %")
print("")
print(f"\nSelf employed and MH diagnosed: {MS:.2f} %")
print(f"\nSelf employed and MH not diagnosed: {NMS:.2f} %")

pd.DataFrame(table, index=setSelf_employedYesNo, columns=setDiagnosedByMedicalYesNo)


Not Self employed and MH diagnosed: 49.48 %

Not Self employed and MH not diagnosed: 50.52 %


Self employed and MH diagnosed: 51.92 %

Self employed and MH not diagnosed: 48.08 %


,MH diagnosed,MH not diagnosed
not self employed,567,579
self employed,149,138


In [15]:
trace1 = go.Bar(
    y=['self employed', 'not self employed'],
    x=[MS, MNS],
    name='MH diagnosed',
    orientation = 'h',
    marker = dict(
        color = 'rgba(0, 153, 153, 0.6)',
        line = dict(
            color = 'rgba(0, 153, 153, 1)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=['self employed', 'not self employed'],
    x=[NMS, NMNS],
    name='MH not diagnosed',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    title="Working as employees = mental illness?",
    xaxis=dict(title="number of people %"),
    yaxis=dict(
         automargin=True
    ),
    barmode='stack'
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

## Correlation between working from home and mental illness - Contingency table, Stacked bar


Il grafico seguente mostra che non vi è alcuna relazione tra il fatto di lavorare da casa o in azienda e di aver riscontrato una malattia mentale.

In [16]:
#Caricamento dei dati relativi al campo: workRemotely
workRemotely = df['Do you work remotely?']
workRemotelySet = set(workRemotely)

#Caricamento dei dati relativi al campo: diagnosedByMedical
diagnosedByMedical = df['Have you been diagnosed with a mental health condition by a medical professional?'].values
diagnosedByMedicalYesNo = np.full(diagnosedByMedical.shape,"",dtype="object")
for i in range(len(diagnosedByMedical)):
    if(diagnosedByMedical[i]=='No'):
        diagnosedByMedicalYesNo[i] = "MH not diagnosed"
    else:
        diagnosedByMedicalYesNo[i] = "MH diagnosed"
        
setWorkRemotely = np.unique(workRemotely)
setDiagnosedByMedicalYesNo = np.unique(diagnosedByMedicalYesNo)

table = np.full((len(setDiagnosedByMedicalYesNo), len(setWorkRemotely)), np.nan, dtype=int)
for i in range (len(setDiagnosedByMedicalYesNo)):
    for j in range (len(setWorkRemotely)):
        table[i,j] = np.sum((workRemotely==setWorkRemotely[j]) & (diagnosedByMedicalYesNo==setDiagnosedByMedicalYesNo[i]))

freq = np.sum(table, axis=1, keepdims=True) # somma i dipendenti e i lavoratori in proprio
people = freq[0]+freq[1]
t_cond_rows = table / freq

MHA = t_cond_rows[0][0]*100
MHN = t_cond_rows[0][1]*100
MHS = t_cond_rows[0][2]*100

NMHA = t_cond_rows[1][0]*100
NMHN = t_cond_rows[1][1]*100
NMHS = t_cond_rows[1][2]*100

print(f"\nWorking always from home and MH diagnosed: {MHA:.2f} %")
print(f"\nWorking never from home and MH diagnosed: {MHN:.2f} %")
print(f"\nWorking sometimes from home and MH diagnosed: {MHS:.2f} %")
print("")
print(f"\nWorking always from home and MH not diagnosed: {NMHA:.2f} %")
print(f"\nWorking never from home and MH not diagnosed: {NMHN:.2f} %")
print(f"\nWorking sometimes from home and MH not diagnosed: {NMHS:.2f} %")


pd.DataFrame(table, index=setDiagnosedByMedicalYesNo, columns=setWorkRemotely)



Working always from home and MH diagnosed: 23.88 %

Working never from home and MH diagnosed: 21.23 %

Working sometimes from home and MH diagnosed: 54.89 %


Working always from home and MH not diagnosed: 23.99 %

Working never from home and MH not diagnosed: 25.24 %

Working sometimes from home and MH not diagnosed: 50.77 %


,Always,Never,Sometimes
MH diagnosed,171,152,393
MH not diagnosed,172,181,364


In [17]:
top_labels = ['Always', "Sometimes", 'Never']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)']

x_data = [[MHA,MHS,MHN],[NMHA,NMHS,NMHN]]

y_data = ['MH diagnosed','MH not diagnosed']

traces = []
for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    title='Work remotely = mental illness?',
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str("{0:.0f}".format(xd[0])) + '%',
#                             text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str("{0:.0f}".format(xd[i])) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='bar-colorscale')

## Correlation between the history of family health and mental illness - Contingency table, Stacked bar


Il grafico seguente mostra che vi è un maggior rischio di sviluppare un disturbo mentale nei soggetti che hanno avuto casi in famiglia.
In particolare, otre il 60 % di coloro che hanno avuto parenti con disturbi mentali soffre di tali problematiche, mentre solamente il 25 % di coloro che non hanno avuto casi di malattia in famiglia riscontra disordini mentali. 

In [18]:
#Caricamento dei dati relativi al campo: family_history
family_history = df['Do you have a family history of mental illness?'].values
family_historyYesNo = np.full(family_history.shape,"",dtype="object")
for i in range(len(family_history)):
    if(family_history[i]=='No'):
        family_historyYesNo[i] = "no mental illness in family"
    else:
        family_historyYesNo[i] = "mental illness in family"

#Caricamento dei dati relativi al campo: diagnosedByMedical
diagnosedByMedical = df['Have you been diagnosed with a mental health condition by a medical professional?'].values
diagnosedByMedicalYesNo = np.full(diagnosedByMedical.shape,"",dtype="object")
for i in range(len(diagnosedByMedical)):
    if(diagnosedByMedical[i]=='No'):
        diagnosedByMedicalYesNo[i] = "MH not diagnosed"
    else:
        diagnosedByMedicalYesNo[i] = "MH diagnosed"
        
setFamily_historyYesNo = np.unique(family_historyYesNo)
setDiagnosedByMedicalYesNo = np.unique(diagnosedByMedicalYesNo)
table = np.full((len(setFamily_historyYesNo), len(setDiagnosedByMedicalYesNo)), np.nan, dtype=int)
for i in range (len(setFamily_historyYesNo)):
    for j in range (len(setDiagnosedByMedicalYesNo)):
        table[i,j] = np.sum((family_historyYesNo==setFamily_historyYesNo[i]) & (diagnosedByMedicalYesNo==setDiagnosedByMedicalYesNo[j]))

freq = np.sum(table, axis=1, keepdims=True) # somma le famiglie con malattie mentali e quelle non 
people = freq[0]+freq[1]

t_cond_rows = table / freq

MF = t_cond_rows[0][0]*100
NMF = t_cond_rows[0][1]*100
MNF = t_cond_rows[1][0]*100
NMNF = t_cond_rows[1][1]*100

print(f"\nMental illness in family and MH diagnosed: {MF:.2f} %")
print(f"\nMental illness in family and MH not diagnosed: {NMF:.2f} %")
print("")
print(f"\nNo mental illness in family and MH diagnosed: {MNF:.2f} %")
print(f"\nNo mental illness in family and MH not diagnosed: {NMNF:.2f} %")

pd.DataFrame(table, index=setFamily_historyYesNo, columns=setDiagnosedByMedicalYesNo)



Mental illness in family and MH diagnosed: 62.43 %

Mental illness in family and MH not diagnosed: 37.57 %


No mental illness in family and MH diagnosed: 25.82 %

No mental illness in family and MH not diagnosed: 74.18 %


,MH diagnosed,MH not diagnosed
mental illness in family,590,355
no mental illness in family,126,362


In [19]:
trace1 = go.Bar(
    y=['mental illness in family', 'no mental illness in family'],
    x=[MF, MNF],
    name='MH diagnosed',
    orientation = 'h',
    marker = dict(
        color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=['mental illness in family', 'no mental illness in family'],
    x=[NMF, NMNF],
    name='MH not diagnosed',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    title="Am I more likely to get sick if I have family members who have had mental illness?",
    xaxis=dict(title="number of people %"),
    yaxis=dict(
         automargin=True
    ),
    barmode='stack'
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

## Feedback:
In generale il lavoro e’ buono.
Seguono commenti puntuali / spunti di miglioramento.

Number of company employees: le categorie devono essere in ordine, altrimenti il grafico ha poco senso.

Interessante “Geo distribution”.

Frequency for job position: Interessante la heatmap ma e’ dominata dal fatto che le prime due righe sono molto piu’ frequenti delle altre. Dovreste riportare le frequenze relative condizionate per permettere un confronto migliore

## Miglioramenti/Modifiche:
- Company number of employees plot - Pie: Ordinameto delle categorie nella legenda 
- Frequency for job position: Il grafico mostrato in fase di presentazione è già sviluppato sulle frequenze condizionate